# Sparkathon implementation


In [1]:
%pip install groq chromadb sentence-transformers gradio requests sarvamai pydub soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.7/163.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 15.8 MB/s

In [3]:
import json
import chromadb
from sentence_transformers import SentenceTransformer

# Load SentenceTransformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load product JSON
with open("dataset.json", "r") as f:
    products = json.load(f)

# ✅ Initialize persistent ChromaDB client
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="products")

def clean_metadata(metadata: dict) -> dict:
    return {
        k: v for k, v in metadata.items()
        if k != "reviews" and isinstance(v, (str, int, float, bool))
    }

# Index documents
for idx, item in enumerate(products):
    doc = f"""
    Product: {item['product_name']}
    Price: {item['product_price']}
    Details: {item['product_details']}
    Specs: {item['specifications']}
    Rating: {item['rating']}
    Availability: {item['availability']}
    """
    embedding = model.encode(doc).tolist()

    collection.add(
        ids=[str(idx)],
        embeddings=[embedding],
        documents=[doc],
        metadatas=[clean_metadata(item)]
    )


In [4]:
# Fetch all items in the collection (assuming you used 'products' as the name)
collection = client.get_collection(name="products")

# Query with a dummy vector to retrieve all documents
all_items = collection.get()

# Display each item nicely
for i in range(len(all_items['ids'])):
    print(f"🛒 Product ID: {all_items['ids'][i]}")
    print(f"📦 Document:\n{all_items['documents'][i]}")
    print(f"📌 Metadata:\n{all_items['metadatas'][i]}")
    print("-" * 80)


🛒 Product ID: 0
📦 Document:

    Product: Fresh Banana, Each
    Price: $0.29
    Details: Fresh, ripe bananas sold individually.
    Specs: Approximate weight: 0.3-0.5 lbs
    Rating: 4.5
    Availability: In stock
    
📌 Metadata:
{'product_price': '$0.29', 'availability': 'In stock', 'specifications': 'Approximate weight: 0.3-0.5 lbs', 'image_url': 'https://i5.walmartimages.com/asr/12345678-1234-5678-1234-567812345678.jpg', 'rating': 4.5, 'product_details': 'Fresh, ripe bananas sold individually.', 'product_name': 'Fresh Banana, Each'}
--------------------------------------------------------------------------------
🛒 Product ID: 1
📦 Document:

    Product: Fresh Strawberries, 1 lb
    Price: $3.98
    Details: Sweet and juicy strawberries, perfect for snacking or desserts.
    Specs: Weight: 1 lb
    Rating: 4.2
    Availability: In stock
    
📌 Metadata:
{'image_url': 'https://i5.walmartimages.com/asr/23456789-2345-6789-2345-678923456789.jpg', 'product_name': 'Fresh Strawberries, 1

# API Keys Configuration


**Groq API Key:** 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


**Sarvam API Key:** sk_93gwy4ia_ZHqFPOEJunnoB5MIrJlBnEvM


**Sarvam API Documentation:** [Sarvam API Quickstart](https://docs.sarvam.ai/api-reference-docs/getting-started/quickstart)


In [ ]:
import os

os.environ["GROQ_API_KEY"] = " "

os.environ["SARVAM_API_KEY"] = " "


In [6]:
from groq import Groq
from sentence_transformers import SentenceTransformer

# Load your SentenceTransformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Set up Groq client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))
groq_model = "llama3-70b-8192"


In [7]:
def rag_query_groq(user_query, n_results=5):
    # Step 1: Embed query
    query_embedding = model.encode(user_query).tolist()

    # Step 2: Query ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )

    # Step 3: Build context
    context = "\n\n".join(results["documents"][0])

    # Step 4: Format prompt
    prompt = f"""You are a helpful AI shopping assistant.
Use the following product descriptions to answer the user's question.

Product Descriptions:
{context}

User Question: {user_query}
Answer:"""

    # Step 5: Call Groq API
    completion = client.chat.completions.create(
        model=groq_model,
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant that answers product-related queries."},
            {"role": "user", "content": prompt}
        ]
    )

    return completion.choices[0].message.content.strip()


In [8]:
print(rag_query_groq("Suggest a juicy fruit under $2"))
print(rag_query_groq("Which dairy products are under $5 and high rated?"))

Based on our product offerings, I would recommend the Fresh Banana, Each, which is priced at $0.29. Not only is it an affordable option, but it's also a juicy and fresh fruit that's perfect for snacking or adding to your favorite recipes. Plus, it has a high rating of 4.5 from our customers!
Based on the product descriptions, here are the dairy products that are under $5 and have a high rating (4.0 or higher):

1. Great Value Skim Milk, Gallon, 128 fl oz - $3.78 (Rating: 4.2)
2. Great Value 2% Reduced Fat Milk, Gallon, 128 fl oz - $3.88 (Rating: 4.2)
3. Great Value Whole Vitamin D Milk, Gallon, 128 fl oz - $3.98 (Rating: 4.4)

All of these options are under $5 and have a rating of 4.0 or higher.


In [9]:
from datetime import datetime
import json
import requests
import os
import base64
import io
from sarvamai import SarvamAI as SarvamClient

class ShoppingMemory:
    """Advanced memory system to track user's shopping journey"""
    def __init__(self):
        self.cart = []
        self.conversation_history = []
        self.session_start = datetime.now()
        self.current_step = "browsing"  # browsing, selecting, checkout, completed
        self.user_preferences = {}

    def add_to_cart(self, product_info):
        """Add product to cart"""
        existing_item = None
        for item in self.cart:
            if item["product"]["product_name"] == product_info["product_name"]:
                existing_item = item
                break
        if existing_item:
            existing_item["quantity"] += 1
        else:
            self.cart.append({
                "product": product_info,
                "added_at": datetime.now(),
                "quantity": 1
            })

    def remove_from_cart(self, product_name):
        self.cart = [item for item in self.cart if item["product"]["product_name"] != product_name]

    def get_cart_summary(self):
        if not self.cart:
            return "🛒 Your cart is empty."
        total = 0
        summary = "🛒 **Your Cart:**\n\n"
        for i, item in enumerate(self.cart, 1):
            price = float(item["product"]["product_price"].replace("$", ""))
            item_total = price * item["quantity"]
            total += item_total
            summary += f"{i}. **{item['product']['product_name']}**\n"
            summary += f"   • Price: {item['product']['product_price']} x {item['quantity']} = ${item_total:.2f}\n"
            summary += f"   • Rating: {item['product'].get('rating', 'N/A')}\n\n"
        summary += f"💰 **Total: ${total:.2f}**\n"
        summary += f"📦 **Items: {len(self.cart)}**"
        return summary

    def update_step(self, step):
        self.current_step = step

    def add_conversation(self, user_input, ai_response):
        entry = {
            "timestamp": datetime.now(),
            "user": user_input,
            "ai": ai_response,
            "step": self.current_step
        }
        self.conversation_history.append(entry)
        if len(self.conversation_history) > 20:
            self.conversation_history = self.conversation_history[-20:]

    def get_conversation_context(self, last_n=3):
        recent = self.conversation_history[-last_n:] if self.conversation_history else []
        context = ""
        for conv in recent:
            context += f"User: {conv['user']}\nAI: {conv['ai']}\n\n"
        return context

# Initialize shopping memory
shopping_memory = ShoppingMemory()

# Enhanced Sarvam AI integration using official SDK
class SarvamAI:
    """
    Official Sarvam AI SDK integration for:
    - Text-to-Speech (TTS)
    - Speech-to-Text (STT)
    - Translation
    """

    def __init__(self, api_key):
        self.client = SarvamClient(api_subscription_key=api_key)
        self.api_key = api_key

    def text_to_speech(self, text, speaker="Anushka", language="hi-IN"):
        """
        Convert text to speech using Sarvam TTS API
        """
        try:
            response = self.client.text_to_speech.convert(
                text=text,
                target_language_code=language,
                speaker=speaker,
                pitch=0.0,
                pace=1.0,
                loudness=1.0,
                speech_sample_rate=22050,
                enable_preprocessing=True,
                model="bulbul:v2"
            )

            if response and hasattr(response, 'audios') and response.audios:
                # Save audio file
                audio_data = base64.b64decode(response.audios[0])
                output_file = f"tts_output_{datetime.now().strftime('%Y%m%d_%H%M%S')}.wav"
                with open(output_file, "wb") as f:
                    f.write(audio_data)
                print(f"🔊 TTS: Audio saved as '{output_file}'")
                return output_file
            else:
                print("❌ TTS: No audio data received")
                return None

        except Exception as e:
            print(f"❌ TTS Error: {str(e)}")
            return None

    def translate_text(self, text, source_language="en-IN", target_language="hi-IN"):
        """
        Translate text using Sarvam Translation API
        """
        try:
            response = self.client.text.translate(
                input=text,
                source_language_code=source_language,
                target_language_code=target_language,
                speaker_gender="Male",
                mode="formal",
                model="mayura:v1",
                enable_preprocessing=True
            )

            if response and hasattr(response, 'translated_text'):
                return response.translated_text
            else:
                print("❌ Translation: No translated text received")
                return text

        except Exception as e:
            print(f"❌ Translation Error: {str(e)}")
            return text

    def speech_to_text(self, audio_file_path, language="hi-IN"):
        """
        Convert speech to text using Sarvam STT API
        """
        try:
            with open(audio_file_path, "rb") as audio_file:
                response = self.client.speech_to_text.transcribe(
                    file=audio_file,
                    model="saarika:v2.5",
                    language_code=language
                )

            if response and hasattr(response, 'transcript'):
                return response.transcript
            else:
                print("❌ STT: No transcript received")
                return ""

        except Exception as e:
            print(f"❌ STT Error: {str(e)}")
            return ""

# Initialize Sarvam AI
sarvam_ai = SarvamAI(os.getenv("SARVAM_API_KEY"))

class AIShoppingAgent:
    """Enhanced AI Agent for shopping assistance with Sarvam AI integration"""

    def __init__(self, memory, groq_client, collection, sarvam_ai):
        self.memory = memory
        self.groq_client = groq_client
        self.collection = collection
        self.sarvam_ai = sarvam_ai

    def get_system_prompt(self):
        cart_info = self.memory.get_cart_summary()
        current_step = self.memory.current_step
        conversation_context = self.memory.get_conversation_context(3)
        return f"""You are a helpful AI shopping assistant with a natural, conversational personality.

CURRENT SESSION INFO:
- Shopping Step: {current_step}
- {cart_info}

RECENT CONVERSATION:
{conversation_context}

YOUR CAPABILITIES:
1. **Product Discovery**: Help find products using the available inventory
2. **Cart Management**: Add, remove, update quantities
3. **Personalized Recommendations**: Based on user preferences and history
4. **Checkout Assistance**: Guide through purchase process
5. **Multi-language Support**: Can translate responses using Sarvam AI

PERSONALITY GUIDELINES:
- Be friendly, helpful, and conversational
- Use emojis appropriately to make responses engaging
- Ask clarifying questions when needed
- Provide specific product details and recommendations
- Remember what we've discussed previously
- Keep responses concise but informative (max 2-3 sentences usually)"""

    def process_query(self, user_input, use_rag=True, enable_tts=False, translate_to=None, tts_language="hi-IN", tts_speaker="Anushka"):
        """Process user query with optional TTS and translation"""

        # Handle special commands
        if user_input.lower() in ["cart", "show cart", "view cart"]:
            response = self.memory.get_cart_summary()
        elif user_input.lower() in ["checkout", "buy now", "proceed"]:
            response = self.start_checkout()
        elif user_input.lower() in ["clear cart", "empty cart"]:
            self.memory.cart = []
            response = "🛒 Cart cleared! Ready to shop for new items."
        else:
            # Use RAG for product queries
            context = ""
            search_results = None
            if use_rag:
                try:
                    query_embedding = model.encode(user_input).tolist()
                    search_results = self.collection.query(
                        query_embeddings=[query_embedding],
                        n_results=3
                    )
                    context = "\n\n".join(search_results["documents"][0])
                except Exception as e:
                    print(f"RAG search error: {e}")
                    context = "No specific product context available."

            prompt = f"""{self.get_system_prompt()}

AVAILABLE PRODUCTS (if relevant):
{context}

USER INPUT: {user_input}

Instructions: Respond naturally and helpfully. If the user wants to add something to cart, be specific about which product."""

            try:
                completion = self.groq_client.chat.completions.create(
                    model="llama3-70b-8192",
                    messages=[
                        {"role": "system", "content": "You are a helpful, friendly AI shopping assistant."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.7,
                    max_tokens=200
                )
                response = completion.choices[0].message.content.strip()
                self.handle_cart_operations(user_input, response, search_results)
            except Exception as e:
                response = f"❌ Sorry, I encountered an error: {str(e)[:100]}. Please try again."

        # Add to conversation history
        self.memory.add_conversation(user_input, response)

        # Optional translation using Sarvam AI
        if translate_to and translate_to != "en-IN":
            translated_response = self.sarvam_ai.translate_text(response, "en-IN", translate_to)
            response = f"{response}\n\n🌐 Translation: {translated_response}"

        # Optional TTS using Sarvam AI
        if enable_tts:
            text_for_tts = translated_response if translate_to and translate_to != "en-IN" else response
            audio_file = self.sarvam_ai.text_to_speech(text_for_tts, tts_speaker, tts_language)
            if audio_file:
                response += f"\n\n🔊 Audio: {audio_file}"

        return response

    def handle_cart_operations(self, user_input, ai_response, search_results):
        """Handle cart operations based on user input"""
        add_keywords = ["add to cart", "buy this", "purchase", "take this", "add it", "get this", "want this"]
        remove_keywords = ["remove", "delete", "take out", "don't want"]
        user_lower = user_input.lower()

        if any(keyword in user_lower for keyword in add_keywords) and search_results:
            if search_results["metadatas"][0]:
                product_metadata = search_results["metadatas"][0][0]
                self.memory.add_to_cart(product_metadata)
                self.memory.update_step("selecting")
        elif any(keyword in user_lower for keyword in remove_keywords):
            for item in self.memory.cart:
                if item["product"]["product_name"].lower() in user_lower:
                    self.memory.remove_from_cart(item["product"]["product_name"])
                    break

    def start_checkout(self):
        """Start checkout process"""
        if not self.memory.cart:
            return "🛒 Your cart is empty. Let's find some great products first! What are you looking for?"

        self.memory.update_step("checkout")
        checkout_msg = f"""🛍️ **Ready to Checkout!**

{self.memory.get_cart_summary()}

💳 **Checkout Process:**
1. 📍 Shipping address
2. 💸 Payment method
3. 📋 Order confirmation

Let's start! What's your shipping address?"""
        return checkout_msg

# Initialize the AI Shopping Agent with Sarvam AI
shopping_agent = AIShoppingAgent(shopping_memory, client, collection, sarvam_ai)

print("🛒 AI Shopping Assistant Initialized!")
print("✅ Sarvam AI SDK integration ready")
print("✅ TTS, Translation, STT services available")
print("✅ Multi-language support available")
print("✅ Memory and conversation tracking")
print("✅ Cart management")
print("\nReady for shopping interaction! 🎉")
print("\n📚 Sarvam AI Documentation: https://docs.sarvam.ai/api-reference-docs/")

🛒 AI Shopping Assistant Initialized!
✅ Sarvam AI SDK integration ready
✅ TTS, Translation, STT services available
✅ Multi-language support available
✅ Memory and conversation tracking
✅ Cart management

Ready for shopping interaction! 🎉

📚 Sarvam AI Documentation: https://docs.sarvam.ai/api-reference-docs/


In [12]:
import gradio as gr
import tempfile
import os

# Enhanced Gradio interface with Sarvam AI features and modern UI

def gradio_shopping_interface(user_input, enable_tts=False, translate_to="en-IN", tts_language="hi-IN", tts_speaker="Anushka"):
    """
    Main shopping interface function with Sarvam AI features
    """
    response = shopping_agent.process_query(
        user_input,
        use_rag=True,
        enable_tts=enable_tts,
        translate_to=translate_to,
        tts_language=tts_language,
        tts_speaker=tts_speaker
    )
    return response

def get_cart_summary():
    """Get current cart summary"""
    return shopping_memory.get_cart_summary()

def clear_cart():
    """Clear the shopping cart"""
    shopping_memory.cart = []
    return "🛒 Cart cleared! Ready to shop for new items."

def translate_text_demo(text, target_language="hi-IN"):
    """Demo function for Sarvam translation"""
    try:
        translated = sarvam_ai.translate_text(text, "en-IN", target_language)
        return f"Original: {text}\n\nTranslated ({target_language}): {translated}"
    except Exception as e:
        return f"Translation error: {str(e)}"

def generate_tts_demo(text, speaker="Anushka", language="hi-IN"):
    """Demo function for Sarvam TTS"""
    try:
        audio_file = sarvam_ai.text_to_speech(text, speaker, language)
        if audio_file:
            return f"🔊 Audio generated: {audio_file}", audio_file
        else:
            return "❌ Failed to generate audio", None
    except Exception as e:
        return f"TTS error: {str(e)}", None

def process_audio_input(audio_file, language="hi-IN"):
    """Process uploaded audio for STT"""
    if audio_file is None:
        return "No audio file provided"

    try:
        transcript = sarvam_ai.speech_to_text(audio_file, language)
        if transcript:
            # Process the transcript as a query
            response = shopping_agent.process_query(transcript, use_rag=True)
            return f"🎙️ Transcript: {transcript}\n\n🤖 Response: {response}"
        else:
            return "❌ Could not transcribe audio"
    except Exception as e:
        return f"STT error: {str(e)}"

# Custom CSS for modern look
custom_css = """
.gradio-container {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
}

.main-header {
    text-align: center;
    padding: 2rem;
    background: rgba(255, 255, 255, 0.1);
    border-radius: 15px;
    margin-bottom: 2rem;
    backdrop-filter: blur(10px);
}

.feature-box {
    background: rgba(255, 255, 255, 0.1);
    border-radius: 10px;
    padding: 1rem;
    margin: 0.5rem;
    backdrop-filter: blur(5px);
}

.btn-primary {
    background: linear-gradient(45deg, #ff6b6b, #ff8e8e);
    border: none;
    border-radius: 25px;
    padding: 12px 30px;
    font-weight: 600;
    transition: all 0.3s ease;
}

.btn-primary:hover {
    transform: translateY(-2px);
    box-shadow: 0 10px 25px rgba(255, 107, 107, 0.3);
}
"""

# Create the enhanced Gradio interface
with gr.Blocks(title="🛒 Sarvam AI Shopping Assistant", theme=gr.themes.Soft(), css=custom_css) as demo:

    gr.HTML("""
    <div class="main-header">
        <h1 style="color: white; font-size: 3rem; margin-bottom: 1rem;">🛒 Sarvam AI Shopping Assistant</h1>
        <h3 style="color: rgba(255,255,255,0.9); font-weight: 300;">
            Experience the future of shopping with AI-powered voice assistance
        </h3>
        <div style="display: flex; justify-content: center; gap: 20px; margin-top: 2rem;">
            <div class="feature-box">🔍 Smart Product Search</div>
            <div class="feature-box">🗣️ Voice Commands</div>
            <div class="feature-box">🌐 Multi-language Support</div>
            <div class="feature-box">🛒 Cart Management</div>
        </div>
    </div>
    """)

    with gr.Tab("🛍️ Shopping Assistant"):
        with gr.Row():
            with gr.Column(scale=2):
                user_input = gr.Textbox(
                    lines=3,
                    placeholder="💬 Ask me anything: 'Show me fruits under $2' or 'Add apple to cart'",
                    label="Your Message",
                    container=True
                )

                with gr.Row():
                    enable_tts = gr.Checkbox(label="🔊 Enable Voice Response", value=False)
                    translate_to = gr.Dropdown(
                        choices=["en-IN", "hi-IN", "bn-IN", "ta-IN", "te-IN", "mr-IN", "gu-IN", "kn-IN"],
                        value="en-IN",
                        label="🌐 Response Language"
                    )

                with gr.Row():
                    tts_speaker = gr.Dropdown(
                        choices=["Anushka", "Manisha", "Vidya", "Arya", "Abhilash", "Karun", "Hitesh"],
                        value="Anushka",
                        label="🎤 Voice Speaker"
                    )
                    tts_language = gr.Dropdown(
                        choices=["hi-IN", "bn-IN", "ta-IN", "te-IN", "mr-IN", "gu-IN", "kn-IN", "en-IN"],
                        value="hi-IN",
                        label="🗣️ Voice Language"
                    )

                submit_btn = gr.Button("Send Message 📤", variant="primary", size="lg")

            with gr.Column(scale=1):
                cart_btn = gr.Button("🛒 View Cart", variant="secondary", size="lg")
                clear_btn = gr.Button("🗑️ Clear Cart", variant="stop", size="lg")

        response_output = gr.Textbox(
            lines=8,
            label="🤖 Assistant Response",
            interactive=False,
            container=True
        )

        cart_output = gr.Textbox(
            lines=6,
            label="🛒 Shopping Cart",
            interactive=False,
            value=shopping_memory.get_cart_summary(),
            container=True
        )

    with gr.Tab("🎙️ Voice Shopping"):
        gr.Markdown("### 🗣️ Shop with Your Voice!")

        with gr.Row():
            with gr.Column():
                audio_input = gr.Audio(
                    type="filepath",
                    label="🎤 Upload Audio File or Record",
                    interactive=True
                )

                stt_language = gr.Dropdown(
                    choices=["hi-IN", "bn-IN", "ta-IN", "te-IN", "mr-IN", "gu-IN", "kn-IN", "en-IN"],
                    value="hi-IN",
                    label="🗣️ Audio Language"
                )

                process_audio_btn = gr.Button("🎙️ Process Voice Command", variant="primary", size="lg")

            with gr.Column():
                stt_output = gr.Textbox(
                    lines=8,
                    label="🎙️ Voice Processing Result",
                    interactive=False
                )

        gr.Markdown("### 📝 Try saying:")
        gr.Markdown("""
        - "मुझे दो डॉलर से कम फल दिखाओ" (Show me fruits under $2)
        - "सेब को कार्ट में डालें" (Add apple to cart)
        - "मेरा कार्ट दिखाओ" (Show my cart)
        """)

    with gr.Tab("🔊 Text-to-Speech Demo"):
        gr.Markdown("### 🔊 Convert Text to Speech")

        with gr.Row():
            with gr.Column():
                tts_text = gr.Textbox(
                    lines=3,
                    placeholder="Enter text to convert to speech...",
                    label="Text Input"
                )

                with gr.Row():
                    tts_demo_speaker = gr.Dropdown(
                        choices=["Anushka", "Manisha", "Vidya", "Arya", "Abhilash", "Karun", "Hitesh"],
                        value="Anushka",
                        label="🎤 Voice"
                    )
                    tts_demo_language = gr.Dropdown(
                        choices=["hi-IN", "bn-IN", "ta-IN", "te-IN", "mr-IN", "gu-IN", "kn-IN", "en-IN"],
                        value="hi-IN",
                        label="🗣️ Language"
                    )

                tts_btn = gr.Button("🔊 Generate Speech", variant="primary", size="lg")

            with gr.Column():
                tts_output = gr.Textbox(
                    lines=4,
                    label="TTS Result",
                    interactive=False
                )

                tts_audio_output = gr.Audio(
                    label="🔊 Generated Audio",
                    interactive=False
                )

    with gr.Tab("🌐 Translation Demo"):
        gr.Markdown("### 🌐 Multi-language Translation")

        with gr.Row():
            with gr.Column():
                trans_text = gr.Textbox(
                    lines=3,
                    placeholder="Enter text to translate...",
                    label="Input Text"
                )
                trans_language = gr.Dropdown(
                    choices=["hi-IN", "bn-IN", "ta-IN", "te-IN", "mr-IN", "gu-IN", "kn-IN", "en-IN"],
                    value="hi-IN",
                    label="Target Language"
                )

                trans_btn = gr.Button("🌐 Translate", variant="primary", size="lg")

            with gr.Column():
                trans_output = gr.Textbox(
                    lines=6,
                    label="Translation Result",
                    interactive=False
                )

    # Event handlers
    submit_btn.click(
        fn=gradio_shopping_interface,
        inputs=[user_input, enable_tts, translate_to, tts_language, tts_speaker],
        outputs=response_output
    )

    cart_btn.click(
        fn=get_cart_summary,
        outputs=cart_output
    )

    clear_btn.click(
        fn=clear_cart,
        outputs=cart_output
    )

    process_audio_btn.click(
        fn=process_audio_input,
        inputs=[audio_input, stt_language],
        outputs=stt_output
    )

    tts_btn.click(
        fn=generate_tts_demo,
        inputs=[tts_text, tts_demo_speaker, tts_demo_language],
        outputs=[tts_output, tts_audio_output]
    )

    trans_btn.click(
        fn=translate_text_demo,
        inputs=[trans_text, trans_language],
        outputs=trans_output
    )

    # Auto-refresh cart display after each interaction
    submit_btn.click(
        fn=get_cart_summary,
        outputs=cart_output
    )

# Launch the interface
demo.launch(
    share=True,
    server_name="0.0.0.0",
    server_port=7860,
    show_error=True,
    inbrowser=True
)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bbb0903afe428c30eb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# 🧪 Test Sarvam AI Integration

print("🧪 Testing Sarvam AI SDK Integration...")
print("=" * 60)

# Test 1: Translation
print("\n🌐 Testing Translation Service:")
test_text = "Welcome to our AI shopping assistant!"
try:
    translated = sarvam_ai.translate_text(test_text, "en-IN", "hi-IN")
    print(f"Original: {test_text}")
    print(f"Hindi Translation: {translated}")
except Exception as e:
    print(f"Translation error: {e}")

# Test 2: Text-to-Speech
print("\n🔊 Testing Text-to-Speech Service:")
try:
    tts_result = sarvam_ai.text_to_speech("नमस्ते! AI शॉपिंग असिस्टेंट में आपका स्वागत है।", "Anushka", "hi-IN")
    if tts_result:
        print(f"✅ TTS successful: {tts_result}")
    else:
        print("❌ TTS failed - check API key and connection")
except Exception as e:
    print(f"TTS error: {e}")

# Test 3: Shopping Assistant
print("\n🛍️ Testing Shopping Assistant:")
test_query = "Show me fruits under $2"
try:
    response = shopping_agent.process_query(test_query, enable_tts=False, translate_to="hi-IN")
    print(f"Query: {test_query}")
    print(f"Response: {response}")
except Exception as e:
    print(f"Shopping assistant error: {e}")

print("\n✅ Sarvam AI SDK Integration Test Complete!")
print("\n📚 Available Sarvam AI Services:")
print("• Text-to-Speech (TTS) - Multiple Indian languages and voices")
print("• Translation - English to/from Indian languages")
print("• Speech-to-Text (STT) - Real-time transcription")
print("\n🚀 Ready to launch enhanced Gradio interface!")

# 🚀 Launch Your AI Shopping Assistant


Your complete AI shopping assistant is now ready! Here's what you can do:


## Features:
- 💬 **Text Chat**: Type your queries for quick responses  
- 🧠 **Smart Memory**: Remembers your preferences and cart items
- 🔍 **Product Search**: Uses RAG to find relevant products
- 🛒 **Cart Management**: Add, view, and manage items
- 💳 **Checkout Process**: Complete guided purchase flow


## How to Use:
1. **Run all cells above** to initialize the system
2. **Execute the cell below** to start shopping
3. **Interact with the Gradio interface**
4. **Shop, add to cart, and checkout!**


## Example Commands:
- "Show me fruits under $2"
- "Add the best apple to my cart"
- "What's in my cart?"
- "I'm ready to checkout"


Ready to start shopping? Run the cell below! 👇


In [ ]:
# 🎉 LAUNCH YOUR SARVAM AI-POWERED SHOPPING ASSISTANT!

print("🛒 Sarvam AI Shopping Assistant Ready!")
print("=" * 70)
print()
print("🚀 **ENHANCED FEATURES WITH SARVAM AI SDK:**")
print("• 🔊 High-quality Text-to-Speech (7 voices: Anushka, Manisha, Vidya, etc.)")
print("• 🌐 Real-time Translation (12+ Indian languages)")
print("• 🎙️ Speech-to-Text with voice commands")
print("• 🛍️ Voice-enabled shopping experience")
print("• 🎨 Beautiful modern UI with animations")
print()
print("📋 **COMPLETE SYSTEM INCLUDES:**")
print("✅ RAG-powered intelligent product search")
print("✅ Advanced shopping cart with memory")
print("✅ Conversational AI with context awareness")
print("✅ Official Sarvam AI SDK integration")
print("✅ Multi-tab interactive Gradio interface")
print("✅ Real-time voice interaction")
print()
print("🌟 **VOICE COMMANDS YOU CAN TRY:**")
print("• 'मुझे दो डॉलर से कम फल दिखाओ' (Hindi)")
print("• 'Show me fruits under $2' (English)")
print("• 'সেব কার্টে যোগ করুন' (Bengali)")
print("• 'கார்ட்டில் ஆப்பிள் சேர்க்கவும்' (Tamil)")
print()
print("🎯 **HOW TO USE:**")
print("1. Run all cells above to initialize the system")
print("2. The Gradio interface will launch automatically")
print("3. Use text, voice, or translation features")
print("4. Shop with AI assistance in your preferred language!")
print()
print("📚 **Sarvam AI Documentation:**")
print("https://docs.sarvam.ai/api-reference-docs/")
print()
print("🎊 **Ready for the ultimate AI shopping experience!**")